In [ ]:
# from torch.utils.data import Dataset


# class ManiSkillDataset(Dataset):
#     def __init__(
#         self,
#         *,
#         pack_root,
#         tasks=None,
#     ):
#         if tasks is None:
#             tasks = ["PegInsertionSide-v1"]

#         paths = [
#             {
#                 "treespec": f"{pack_root}/{task}/treespec.pkl",
#                 "traj_lengths": f"{pack_root}/{task}/traj_lengths.npy",
#                 "episode_format": f"{pack_root}/{task}/episode_%d.npz",
#             }
#             for task in tasks
#         ]

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         return self.data[idx]